In [171]:
import pandas as pd

In [191]:
agg_ops = ['', 'MAX', 'MIN', 'COUNT', 'SUM', 'AVG']
cond_ops = ['=', '>', '<', 'OP']

def expand_query(row):
    d = row['sql']
    
    sel_index  = d['sel']
    agg_index  = d['agg']
    conditions = d['conds']
    
    rep = 'SELECT {agg} {sel} FROM table'.format(
            agg=agg_ops[agg_index],
            sel=row['header'][sel_index])
    if conditions:
                rep += ' WHERE ' + ' AND '.join(['{} {} {}'.format(row['header'][i], cond_ops[o], v) 
                                                 for i, o, v in conditions])
    
    return rep

def quote(row):
    if row['conds'] == []:
        return row
    x = row['conds'][0]
    if isinstance(x[2], str) and ' ' in x[2]:
        return {'sel': row['sel'], 'conds':   [[x[0], x[1], f'"{x[2]}"']], 'agg': row['agg']}
    else:
        return row

In [192]:
df        = pd.read_json('~/WIkiSQL/data/dev.jsonl', lines=True)
df_tables = pd.read_json('~/WIkiSQL/data/dev.tables.jsonl', lines=True)
df        = df.merge(df_tables[['id','header']], left_on='table_id', right_on='id')

df['header'] = df['header'].apply(lambda x: [f'"{i}"' if ' ' in i else i for i in x])
df['query']  = df.apply(lambda x: expand_query(x), axis=1)
df['sql']    = df['sql'].apply(lambda x: quote(x))

In [193]:
print(df['query'].loc[0])

SELECT  Position FROM table WHERE "School/Club Team" = Butler CC (KS)


In [196]:
print(df.loc[0]['sql']['conds'])

[[5, 0, '"Butler CC (KS)"']]


In [15]:
df.columns.to_list()

['phase', 'table_id', 'question', 'sql', 'id', 'header']

0       SELECT  Position FROM table WHERE School/Club ...
1       SELECT COUNT School/Club Team FROM table WHERE...
2       SELECT  School/Club Team FROM table WHERE No. ...
3              SELECT  Player FROM table WHERE No. = '42'
4       SELECT  Player FROM table WHERE Position = 'Gu...
                              ...                        
8416    SELECT  Loss FROM table WHERE Attendance > '49...
8417    SELECT MAX Attendance FROM table WHERE Opponen...
8418    SELECT  Party FROM table WHERE Representative ...
8419    SELECT  State FROM table WHERE Representative ...
8420    SELECT  Lifespan FROM table WHERE Party = 'dem...
Length: 8421, dtype: object